In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.layers import Flatten, Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator , img_to_array, load_img
from keras.applications.mobilenet import MobileNet, preprocess_input 
from keras.losses import categorical_crossentropy

In [ ]:
# Working with pre trained model 

base_model = MobileNet( input_shape=(224,224,3), include_top= False )

for layer in base_model.layers:
  layer.trainable = False


x = Flatten()(base_model.output)
x = Dense(units=7 , activation='softmax' )(x)

# creating our model.
model = Model(base_model.input, x)

In [ ]:
model.compile(optimizer='adam', loss= categorical_crossentropy , metrics=['accuracy']  )

In [ ]:
import os
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

In [ ]:
sessions = {}
directory_template = './data/mmi/Sessions/$session$/'
for session in os.listdir('./data/mmi/Sessions'):
    # Find XML filename
    directory = directory_template.replace('$session$', session)
    files = os.listdir(directory)
    xml_filename = directory + files[1]
    
    # Load AUs
    aus = []
    root = ET.parse(xml_filename).getroot()
    for au in root.iter('ActionUnit'):
        aus.append(au.attrib['Number'])
    
    # Load JPG filename
    jpg_filename = directory + files[0]
    
    sessions[session] = {
        'aus': aus,
        'img': jpg_filename
    }
    
assert len(sessions) == 493
print(f'{len(sessions)} sessions loaded')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_sessions = pd.DataFrame.from_dict(sessions, orient='index')
df_sessions.index.rename('session', inplace=True)

df_sessions_null = df_sessions.loc[df_sessions['aus'].apply(lambda x: len(x)) == 0]
df_sessions = df_sessions.loc[df_sessions['aus'].apply(lambda x: len(x)) != 0]

print(f'df_sessions_null: {df_sessions_null.shape}')
print(f'df_sessions: {df_sessions.shape}')
print()
df_sessions.info()
print()
df_sessions.head()

In [ ]:
train_datagen = ImageDataGenerator(
     zoom_range = 0.2, 
     shear_range = 0.2, 
     horizontal_flip=True, 
     rescale = 1./255
)

train_data = train_datagen(
  dataframe = df_sessions,
  directory="./train/",
  x_col = "id",
  y_col = "label",
  subset = "training",
  batch_size = 32,
  seed = 42,
  shuffle = True,
  class_mode = "categorical",
  target_size = (32,32))

In [ ]:
val_datagen = ImageDataGenerator(rescale = 1./255 )

val_data = val_datagen.flow_from_directory(directory= "/content/test", 
                                           target_size=(224,224), 
                                           batch_size=32,
                                  )

In [ ]:
# to visualize the images in the traing data denerator 

t_img , label = train_data.next()

#-----------------------------------------------------------------------------
# function when called will prot the images 
def plotImages(img_arr, label):
  """
  input  :- images array 
  output :- plots the images 
  """
  count = 0
  for im, l in zip(img_arr,label) :
    plt.imshow(im)
    plt.title(im.shape)
    plt.axis = False
    plt.show()
    
    count += 1
    if count == 10:
      break

#-----------------------------------------------------------------------------
# function call to plot the images 
plotImages(t_img, label)

In [ ]:
hist = model.fit_generator(train_data, 
                           steps_per_epoch= 10, 
                           epochs= 30, 
                           validation_data= val_data, 
                           validation_steps= 8)

In [ ]:
# Loading the best fit model 
from keras.models import load_model
model = load_model("/content/best_model.h5")